In [7]:
import time
import pandas as pd
import numpy as np
import datetime
import warnings
warnings.filterwarnings("ignore")
import re
from glob import glob
from tqdm import tqdm
from wordcloud import WordCloud 
# get_ipython().run_line_magic('matplotlib', 'inline')
from matplotlib import font_manager, rc
import platform
import seaborn as sns
import matplotlib
import matplotlib.pyplot as plt
from konlpy.tag import Okt

from collections import Counter
import missingno
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.feature_extraction.text import TfidfVectorizer, CountVectorizer
from sklearn.preprocessing import MinMaxScaler

In [8]:
data_df = pd.read_excel('./data_df_full (Mid_Project_DataProcessing_(2)).xlsx')

In [9]:
data_df.head()

,이름,이미지,링크,가격,분류,리뷰건수,등록일,찜하기,급여대상,중량,주원료,등급,기능,입자크기
0,내추럴발란스 LID 고구마 연어 건식사료 작은입자,https://shopping-phinf.pstatic.net/main_144180...,https://cr.shopping.naver.com/adcr.nhn?x=r1RXI...,32000,건식사료,3250,2018.05.,101,어덜트,6120,"고구마, 연어",그레인프리,"영양공급, 피부/털개선, 저알러지, 면역력강화, 눈물개선/눈건강",작은 8mm 미만
1,ANF 램 홀리스틱 프리미엄 램28 15kg 10kg 양고기 애견사료,https://shopping-phinf.pstatic.net/main_819317...,https://cr.shopping.naver.com/adcr.nhn?x=M%2Bj...,49850,건식사료,1675,2021.09.,182,"어덜트, 대형견",15000,"양, 쌀",홀리스틱(1등급),"영양공급, 뼈/관절강화, 피부/털개선, 체중유지",보통 8~13mm
2,아스쿠 펠리쿠치나 1.2kg 3kg 반습식 강아지 사료 소고기 오리 연어,https://shopping-phinf.pstatic.net/main_815662...,https://cr.shopping.naver.com/adcr.nhn?x=RPCfu...,20000,소프트사료,1752,2019.01.,89,"어덜트, 시니어, 임신/수유, 대형견",1200,"소, 오리, 연어",그레인프리,영양공급,작은 8mm 미만
3,강아지사료 20kg 프라임 블랙에디션 25 34 퍼포먼스 애견 특수견 진도 대용량 ...,https://shopping-phinf.pstatic.net/main_821120...,https://cr.shopping.naver.com/adcr.nhn?x=T8bj4...,26500,건식사료,5660,2019.06.,277,"퍼피, 어덜트, 임신/수유, 대형견",20000,"닭, 곡물",프리미엄(3등급),"영양공급, 퍼포먼스",보통 8~13mm
4,하림 더리얼 그레인프리 크런치 오븐베이크드 모음전,https://shopping-phinf.pstatic.net/main_823923...,https://cr.shopping.naver.com/adcr.nhn?x=ms%2B...,17100,건식사료,9436,2020.03.,401,"퍼피, 어덜트, 시니어, 임신/수유",1000,"닭, 양, 연어",그레인프리,"영양공급, 피부/털개선, 저알러지, 인도어, 식욕증진(기호성)",보통 8~13mm


In [10]:
ingredients = set(",".join(data_df['주원료']).replace(" ","").split(','))

data_df_ing = pd.DataFrame(columns = list(ingredients))

for i in range(len(data_df['주원료'])):
    new_tuple = dict(zip(list(ingredients), [0]*len(list(ingredients)))) 
    for ing_have in data_df['주원료'][i].replace(" ","").split(','):
        for ing_item in list(ingredients):
            if ing_have == ing_item:
                new_tuple[ing_item]=1
    data_df_ing = data_df_ing.append(new_tuple, ignore_index=True)
    
            

def ingredient_scoring(query):
    tokenizer = Okt()

    del_list = ['kg'] 

    raw_pos_tagged = tokenizer.pos(query, norm=True, stem=True)
    cleaned_query = []
    for word in raw_pos_tagged:
        if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign", "Number"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
            if (word[0] not in del_list): # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
                cleaned_query.append(word[0])

    cleaned_query = " ".join(cleaned_query)
    
    querys = cleaned_query.split(' ')
    print(querys)
    
    query_tuple = dict(zip(list(ingredients), [0]*len(list(ingredients))))

    for query in querys:
        for ing_item in list(ingredients):
            if query in ing_item:
                query_tuple[ing_item]=1
            # 닭고기, 소고기,양고기,밀가루,쌀알 등 단어가 들어올 시 한글자 인식 불가
            if ing_item in query:
                query_tuple[ing_item]=1
    
    ing_scores = []
    
    for i in range(len(data_df_ing)):
        score = 0
        for ing in list(data_df_ing.columns):
            if query_tuple[ing]==1 & data_df_ing[ing][i]==1:
                score +=1
        ing_scores.append(score)
        
    result = pd.concat([data_df, pd.DataFrame(ing_scores, columns = ['score'])], axis = 1)
    result = result[result['score']>=1].sort_values('score',ascending=False)
    
    return result

In [11]:
ingredient_scoring('닭가슴살 사료')

['닭', '가슴', '살', '사료']


,이름,이미지,링크,가격,분류,리뷰건수,등록일,찜하기,급여대상,중량,주원료,등급,기능,입자크기,score
3,강아지사료 20kg 프라임 블랙에디션 25 34 퍼포먼스 애견 특수견 진도 대용량 ...,https://shopping-phinf.pstatic.net/main_821120...,https://cr.shopping.naver.com/adcr.nhn?x=T8bj4...,26500,건식사료,5660,2019.06.,277,"퍼피, 어덜트, 임신/수유, 대형견",20000,"닭, 곡물",프리미엄(3등급),"영양공급, 퍼포먼스",보통 8~13mm,1
627,웰니스 컴플리트헬스 스몰브리드 시니어 애견사료 1.8kg,https://shopping-phinf.pstatic.net/main_822060...,https://cr.shopping.naver.com/adcr.nhn?x=Q%2Bj...,26000,건식사료,110,2022.03.,1,시니어,1800,"닭, 칠면조",홀리스틱(1등급),"뼈/관절강화, 다이어트/중성화",보통 8~13mm,1
625,딩고나투라 퍼피앤주니어 강아지 가수분해 사료 비숑 말티즈 푸들 애완견 관절 사료 3kg,https://shopping-phinf.pstatic.net/main_835849...,https://cr.shopping.naver.com/adcr.nhn?x=lnOOJ...,32300,건식사료,5,2021.11.,1,퍼피,3000,"돼지, 닭",슈퍼프리미엄(2등급),"영양공급, 피부/털개선, 저알러지, 면역력강화, 소화/장기능개선",작은 8mm 미만,1
622,웰니스 코어 퍼피 포뮬라 1.8kg 강아지사료,https://shopping-phinf.pstatic.net/main_813644...,https://cr.shopping.naver.com/adcr.nhn?x=L56SQ...,22250,건식사료,29,2021.03.,2,"퍼피, 임신/수유",1800,"닭, 칠면조, 연어",그레인프리,"영양공급, 뼈/관절강화, 피부/털개선, 저알러지, 면역력강화",보통 8~13mm,1
620,(강아지) 로얄캐닌 독 스킨케어 스몰독 2kg,https://shopping-phinf.pstatic.net/main_828467...,https://cr.shopping.naver.com/adcr.nhn?x=0DSQh...,29000,건식사료,7,2022.04.,4,"어덜트, 시니어, 임신/수유, 대형견",2000,"닭, 곡물",슈퍼프리미엄(2등급),"피부/털개선, 저알러지, 처방식",보통 8~13mm,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
313,오리젠 피트앤트림 독 2kg+봉제인형+와그작하루간식 (강아지사료),https://shopping-phinf.pstatic.net/main_821701...,https://cr.shopping.naver.com/adcr.nhn?x=4IR2k...,34500,건식사료,73,2019.11.,1,"퍼피, 어덜트, 시니어",2000,"닭, 칠면조, 생선/해산물",그레인프리,"영양공급, 피부/털개선, 저알러지, 다이어트/중성화, 체중유지",큰 13mm 이상,1
312,웰니스 컴플리트헬스 그레인프리 스몰브리드 5.4kg 강아지사료,https://shopping-phinf.pstatic.net/main_813622...,https://cr.shopping.naver.com/adcr.nhn?x=q9FpH...,50810,건식사료,23,2021.03.,2,어덜트,5000,"닭, 칠면조, 연어",그레인프리,"치석제거, 영양공급, 뼈/관절강화, 피부/털개선, 저알러지",보통 8~13mm,1
310,로얄캐닌 미디엄 퍼피 새끼강아지 사료 4kg 10kg,https://shopping-phinf.pstatic.net/main_834397...,https://cr.shopping.naver.com/adcr.nhn?x=EbvUC...,29900,건식사료,31,2021.09.,7,"퍼피, 대형견",4000,"돼지, 닭, 칠면조",홀리스틱(1등급),"영양공급, 뼈/관절강화, 면역력강화, 소화/장기능개선, 두뇌발달",보통 8~13mm,1
307,메라바이탈 웨이트컨트롤 강아지 처방 다이어트 관절 당뇨 위장 가수분해 알러지 동물병...,https://shopping-phinf.pstatic.net/main_838749...,https://cr.shopping.naver.com/adcr.nhn?x=SDoQF...,45800,건식사료,123,2022.02.,144,"어덜트, 시니어, 대형견",3000,"닭, 오리, 칠면조",그레인프리,"뼈/관절강화, 저알러지, 다이어트/중성화, 처방식, 체중유지",보통 8~13mm,1


# 유사도 가중합산

In [12]:
# 복수값을 허용하는 열에 대해 one-hot-encoding
def feat_to_vec(data_df, col):
    items = set(",".join(data_df[col]).replace(" ","").split(','))
    data_df_col = pd.DataFrame(columns = list(items))

    for i in range(len(data_df[col])):
        new_tuple = dict(zip(list(items), [0]*len(list(items)))) 
        for item_have in data_df[col][i].replace(" ","").split(','):
            for item_candidate in list(items):
                if item_have == item_candidate:
                    new_tuple[item_candidate]=1
        data_df_col = data_df_col.append(new_tuple, ignore_index=True)
        
    return data_df_col


# 토큰화 및 불용어 제거한 corpus 생성

def make_cleaned_corpus(str):
    tokenizer = Okt()
    del_list = ['kg'] 
    raw_pos_tagged = tokenizer.pos(str, norm=True, stem=True)
    word_cleaned = []
    for word in raw_pos_tagged:
        if not word[1] in ["Josa", "Eomi", "Punctuation", "Foreign", "Number"]: # Foreign == ”, “ 와 같이 제외되어야할 항목들
            if word[0] not in del_list: # 한 글자로 이뤄진 단어들을 제외 & 원치 않는 단어들을 제외
                word_cleaned.append(word[0])
    result = " ".join(word_cleaned)
    return result


cleaned_name = []
for i in range(len(data_df['이름'])):
    word_cleaned = make_cleaned_corpus(data_df['이름'][i])
    cleaned_name.append(word_cleaned)
    
# 이름열 기반으로 tfidf_vectorizer fitting 

tfidf_vectorizer = TfidfVectorizer()
tfidf_vectorizer.fit(cleaned_name) # 벡터라이저가 단어들을 학습합니다.
tfidf_matrix = tfidf_vectorizer.transform(cleaned_name).todense()

data_df_func = feat_to_vec(data_df,'기능')
data_df_target = feat_to_vec(data_df,'급여대상')
data_df_ingredient = feat_to_vec(data_df,'주원료')
data_df_cat = feat_to_vec(data_df,'분류')
data_df_grade =  feat_to_vec(data_df,'등급')

for i in range(len(data_df_target)):
    if data_df_target['전연령'][i]==1:
        data_df_target['시니어'][i]=1
        data_df_target['퍼피'][i]=1
        data_df_target['어덜트'][i]=1           
        
data_df_target = data_df_target.drop('전연령', axis=1)


In [13]:
def get_score_on_col(query, data_df_col):
    cleaned_query = make_cleaned_corpus(query)
    querys = cleaned_query.split(' ')
    items = list(data_df_col.columns)
    query_tuple = dict(zip(items, [0]*len(items)))
    
    for query in querys:
        for item in items:
            if query in item:
                query_tuple[item]=1 # 두번 입력해도 가중 X
    
    scores = []
    for i in range(len(data_df_col)):
        score = 0
        for item in items:
            if query_tuple[item]==1 & data_df_col[item][i]==1:
                score +=1
        scores.append(score)
        
        scaler = MinMaxScaler()
        result = scaler.fit_transform(np.array(scores).reshape(len(scores),1))
        result = result.reshape(len(result))
    return result



In [14]:
query = "그레인프리 연어맛 어덜트사료"

def dogfood_search(query):
    cleaned_query = make_cleaned_corpus(query)
    query_tfidf = tfidf_vectorizer.transform([cleaned_query]).todense()

    name_cosine_sim = cosine_similarity(query_tfidf, tfidf_matrix)[0]
    func_scores = get_score_on_col(query, data_df_func)
    target_scores = get_score_on_col(query, data_df_target)
    ingredient_scores = get_score_on_col(query, data_df_ingredient)
    cat_scores = get_score_on_col(query, data_df_cat)
    grade_scores = get_score_on_col(query, data_df_grade)


    a = 5 # 이름 가중치
    b = 1 # 기능 가중치
    c = 1 # 급여대상 가중치
    d = 1 # 주원료 가중치
    e = 1 # 분류 가중치
    f = 1 # 등급 가중치
    total_score =(a*name_cosine_sim+b*func_scores+c*target_scores+d*ingredient_scores+e*cat_scores+f*grade_scores)/(a+b+c+d+e+f)
    result = pd.concat([data_df, pd.DataFrame(total_score, columns = ['score'])], axis = 1)
    result = result.sort_values('score',ascending=False)
    
    return result
    
result = dogfood_search(query)

In [15]:
col = '등급'
value = '그레인프리'
result[result[col]==value]

,이름,이미지,링크,가격,분류,리뷰건수,등록일,찜하기,급여대상,중량,주원료,등급,기능,입자크기,score
607,뉴트리웰 그레인프리 1.5kg 연어 오리 강아지사료,https://shopping-phinf.pstatic.net/main_827231...,https://cr.shopping.naver.com/adcr.nhn?x=LL4Si...,10500,건식사료,31,2020.10.,1,"퍼피, 어덜트, 시니어",1500,"오리, 연어, 기타",그레인프리,"냄새제거, 영양공급, 피부/털개선, 저알러지, 면역력강화",보통 8~13mm,0.631290
750,더리얼 그레인프리 크런치 연어 어덜트 1.6kg,https://shopping-phinf.pstatic.net/main_125236...,https://cr.shopping.naver.com/adcr.nhn?x=1%2Ba...,30000,건식사료,427,2020.10.,14,"어덜트, 대형견",1600,"과일/야채, 연어",그레인프리,"영양공급, 저알러지, 면역력강화, 종합비타민, 식욕증진(기호성)",보통 8~13mm,0.591529
593,나우사료 그레인프리 올브리드 어덜트 9.98kg 중대형견 강아지사료,https://shopping-phinf.pstatic.net/main_836319...,https://cr.shopping.naver.com/adcr.nhn?x=0Vwhm...,116100,건식사료,2,2022.03.,2,"어덜트, 대형견",9980,"오리, 칠면조, 연어",그레인프리,"영양공급, 피부/털개선, 소화/장기능개선",보통 8~13mm,0.582618
472,아이누 사료 저알러지 그레인프리 퍼피 어덜트 1.5kg,https://shopping-phinf.pstatic.net/main_810457...,https://cr.shopping.naver.com/adcr.nhn?x=HIelo...,13500,건식사료,236,2019.09.,5,"퍼피, 어덜트",1500,"오리, 고구마, 연어",그레인프리,"영양공급, 저알러지, 면역력강화, 체중유지",보통 8~13mm,0.573672
475,이볼브 그레인프리 독 연어 5.44kg,https://shopping-phinf.pstatic.net/main_823016...,https://cr.shopping.naver.com/adcr.nhn?x=%2BUx...,57000,건식사료,49,2019.12.,1,"퍼피, 어덜트, 시니어, 대형견",5440,연어,그레인프리,"피부/털개선, 눈물개선/눈건강, 식욕증진(기호성)",작은 8mm 미만,0.572410
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
739,이즈칸 주니어 15kg 7kg 새끼 어린 아기 강아지사료 대용량,https://shopping-phinf.pstatic.net/main_829621...,https://cr.shopping.naver.com/adcr.nhn?x=f0%2F...,32800,건식사료,2,2021.02.,2,퍼피,15000,"닭, 과일/야채, 기타",그레인프리,"영양공급, 뼈/관절강화, 피부/털개선, 저알러지, 식욕증진(기호성)",작은 8mm 미만,0.107844
608,"피쉬포독 화이트피쉬 퍼피 1kg, 3.9kg 강아지 눈물사료",https://shopping-phinf.pstatic.net/main_833393...,https://cr.shopping.naver.com/adcr.nhn?x=OIfm7...,22000,건식사료,16,2021.11.,10,퍼피,1000,생선/해산물,그레인프리,"뼈/관절강화, 피부/털개선, 면역력강화, 눈물개선/눈건강, 소화/장기능개선",작은 8mm 미만,0.107037
536,뉴트리소스 미국 애견 사료 코스트코입점 대포장 모음 반려견 전연령 대형견 슈퍼프리미엄급,https://shopping-phinf.pstatic.net/main_814026...,https://cr.shopping.naver.com/adcr.nhn?x=WeBC3...,67900,건식사료,7,2021.06.,1,대형견,13600,"닭, 오리, 쌀",그레인프리,"뼈/관절강화, 피부/털개선, 면역력강화, 퍼포먼스, 소화/장기능개선",보통 8~13mm,0.105275
197,메라 독 퓨어센시티브 주니어 칠면조&쌀 새끼 아기 강아지 2개월 3개월 4개월 눈물...,https://shopping-phinf.pstatic.net/main_838741...,https://cr.shopping.naver.com/adcr.nhn?x=U57k6...,16800,건식사료,132,2022.02.,147,"퍼피, 임신/수유",1000,"칠면조, 쌀",그레인프리,"영양공급, 피부/털개선, 저알러지, 종합비타민, 소화/장기능개선",작은 8mm 미만,0.103811


# 필터링

In [16]:
def filter_result_add(result, col, value):
    result = result[result[col].str.contains(value, na=False)]
    return result # 필터링 단어 추가

In [17]:
def filter_result_except(result, col, value):
    result = result[result[col] != value]
    return result # 입력한 단어만 제외하고 검색하는 필터링

# Sorting

In [18]:
def sort_result_descending(result, col): #내림차순
    result = result.sort_values(col, ascending = False)
    return result

In [19]:
def sort_result_ascending(result, col): #오름차순
    result = result.sort_values(col, ascending = True)
    return result

# 추천 시스템

In [22]:
query = input('검색어를 입력해주세요\n')
result = dogfood_search(query)
print(result.head())

while True:    
    print("새로운 검색을 원하시면 '검색'을 입력해주세요")
    print("종료를 원하시면 '종료'를 입력해주세요")
    print("결과를 정렬하고 싶으시면 '정렬'을 입력해주세요")
    print("결과를 필터링하고 싶으시면 '필터링'을 입력해주세요")
    query = input()
    print()
    if query == '종료':
        break
    elif query == '정렬':
        query_kind = input("정렬 기준을 입력해주세요. (가격, 리뷰건수, 찜하기, 관련도) : ")
        query_order = input("정렬 순서를 입력해주세요. (오름차순, 내림차순) : ")
        if query_kind.strip() not in [ "가격", "리뷰건수", "찜하기", "관련도" ] :
            print('정렬 기준을 잘못 입력하셨습니다. 기본값인 가격 기준으로 정렬된 결과를 출력합니다.')
            query_kind = '가격'
        if query_order.strip() not in [ "오름차순", "내림차순" ] :
            print('정렬 순서를 잘못 입력하셨습니다. 기본값인 오름차순으로 정렬된 결과를 출력합니다.')
            query_order = '오름차순'
        if query_order.strip() == '오름차순':
            result = sort_result_ascending(result, query_kind)
        else: # '내림차순'
            result = sort_result_descending(result, query_kind)
        print(result.head())
    elif query == '필터링':
        query_cat = input("필터링 기준을 입력해주세요. (급여대상, 주원료, 등급, 기능, 입자크기) : ")
        query_pm = input("추가 / 제외 중 하나 선택해주세요. (추가, 제외) : ")
        if query_cat.strip() not in ["급여대상", "주원료", "등급", "기능", "입자크기"] :
            print('필터링 기준을 잘못 입력하셨습니다. 기본값인 급여대상 기준으로 정렬된 결과를 출력합니다.')
            query_cat = "급여대상"
        if query_pm.strip() not in ["추가", "제외"]:
            print("잘못 입력하셨습니다. 기본값인 필터링 추가로 진행합니다.")
            query_pm = "추가"
        if query_pm.strip() == "추가":
            query = input("필터링 하고 싶은 값을 입력해주세요 : ")
            result_add = filter_result_add(result, query_cat, query)
            print(result_add.head())
        else: #필터링 제외
            query = input("필터링 하고 싶은 값을 입력해주세요 : ")
            result_except = filter_result_except(result, query_cat, query)
            print(result_except.head())
    elif query == '검색':
        query = input('검색어를 입력해주세요')
        dogfood_search(query)
    else:
        print("잘못 입력하셨습니다. 다시 입력해주세요")

검색어를 입력해주세요
로얄캐닌
                              이름  \
747            로얄캐닌 독 미니 어덜트 8kg   
785  로얄캐닌 강아지사료 1.5kg 퍼피 시니어 어덜트   
211         로얄캐닌 독 말티즈 어덜트 1.5kg   
746    로얄캐닌 말티즈 어덜트 사료 1.5kg 3kg   
706       (강아지) 로얄캐닌 독 다이아베틱 7kg   

                                                   이미지  \
747  https://shopping-phinf.pstatic.net/main_839470...   
785  https://shopping-phinf.pstatic.net/main_927963...   
211  https://shopping-phinf.pstatic.net/main_839333...   
746  https://shopping-phinf.pstatic.net/main_834275...   
706  https://shopping-phinf.pstatic.net/main_828370...   

                                                    링크      가격    분류  리뷰건수  \
747  https://cr.shopping.naver.com/adcr.nhn?x=rFS0D...   62230  건식사료     6   
785  https://cr.shopping.naver.com/adcr.nhn?x=Wqrhh...   16900  건식사료    11   
211  https://cr.shopping.naver.com/adcr.nhn?x=uVrzz...   17200  건식사료     7   
746  https://cr.shopping.naver.com/adcr.nhn?x=WjeuB...   21900  건식사료    19   
706  https://cr.shopping.na